# Without Pipelines

In [7]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [8]:
# load dataset using seaborn
df = sns.load_dataset('titanic')

In [9]:
# Drop unnecessary columns from the DataFrame
df.drop(columns=['who', 'adult_male', 'deck', 'embark_town', 'alive', 'alone', 'class'], inplace= True)

In [10]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [11]:
# Split the DataFrame into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=["survived"]),  # drop by name
    df["survived"],                 # target values
    test_size=0.2,
    random_state=42
)


In [12]:
X_train.head()

,pclass,sex,age,sibsp,parch,fare,embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [13]:
y_train.head()

331    0
733    0
382    0
704    0
813    0
Name: survived, dtype: int64

In [14]:
# Check for missing values in the DataFrame
df.isnull().sum()

survived      0
pclass        0
sex           0
age         177
sibsp         0
parch         0
fare          0
embarked      2
dtype: int64

In [15]:
# Apply simple imputation

si_age =SimpleImputer()
si_embarked = SimpleImputer(strategy='most_frequent')

X_train_age = si_age.fit_transform(X_train[['age']])
X_train_embarked = si_embarked.fit_transform(X_train[['embarked']])

X_test_age = si_age.transform(X_test[['age']])
X_test_embarked =si_embarked.transform(X_test[['embarked']])


In [16]:
# Onehot encoding sex and embarked
ohe_sex = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
ohe_embarked= OneHotEncoder(sparse_output=False, handle_unknown='ignore')

X_train_sex = ohe_sex.fit_transform(X_train[['sex']])
X_train_embarked = ohe_embarked.fit_transform(X_train[['embarked']])

X_test_sex = ohe_sex.transform(X_test[['sex']])
X_test_embarked = ohe_embarked.transform(X_test[['embarked']])

In [17]:
X_train_embarked

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]])

In [18]:
X_train.head(2)

,pclass,sex,age,sibsp,parch,fare,embarked
331,1,male,45.5,0,0,28.5,S
733,2,male,23.0,0,0,13.0,S


In [19]:
# Remove the original columns that were transformed
X_train_rem =X_train.drop(columns=['sex', 'age', 'embarked'])
X_test_rem = X_test.drop(columns= ['sex', 'age', 'embarked'])

In [28]:
# Concatenate all transformed and remaining features
X_train_transformed = np.concatenate((X_train_rem, X_train_age, X_train_sex, X_train_embarked), axis =1)
X_test_transformed = np.concatenate((X_test_rem, X_test_age, X_test_sex, X_test_embarked), axis =1)

In [21]:
X_test_transformed.shape

(179, 11)

In [29]:
# Create and train the Decision Tree Classifier
clf = DecisionTreeClassifier()
clf.fit(X_train_transformed, y_train)

DecisionTreeClassifier()

In [30]:
# Make predictions on the test set
y_pred= clf.predict(X_test_transformed)
y_pred

array([0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 1])

In [31]:
# Evaluate the model's accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.776536312849162

In [33]:
# Save the trained model using pickle
import pickle
pickle.dump(ohe_sex, open('ohe_sex.pkl', 'wb'))
pickle.dump(ohe_embarked, open('ohe_embarked.pkl', 'wb'))
pickle.dump(clf, open('clf.pkl', 'wb'))